In [1]:
import pandas as pd
import altair as alt

In [2]:
from ecostyles import EcoStyles
# Create styles instance
styles = EcoStyles()
# Register and enable a theme
styles.register_and_enable_theme(theme_name="article")  # or "article"

In [3]:
sector_gva_delta =pd.read_excel('prod_change.xlsx', sheet_name='Data for Dataviz', skiprows=6)

In [4]:
sector_gva_delta.head()

,Industry,GVA,Hours worked,Output per hour worked
0,Whole Economy excluding imputed rental,0.058250,0.028930,0.028496
1,A: Agriculture,0.057186,-0.135857,0.223393
2,B: Mining and Quarrying,-0.384713,-0.138518,-0.285780
3,C: Manufacturing,0.046363,-0.075203,0.131452
4,D: Energy,-0.496503,0.062503,-0.526122


In [5]:
sector_gva_delta['Industry'] = sector_gva_delta['Industry'].str.split(':').str[1].str.strip()
sector_gva_delta['Industry'] = sector_gva_delta['Industry'].where(~sector_gva_delta['Industry'].isna(), 'Whole Economy')

In [10]:
sector_gva_delta['OPHW_tooltip'] = sector_gva_delta['Output per hour worked'].apply(lambda x: f"{x*100:.1f}%")

# Sort the dataframe by Output per hour worked descending for consistent y-axis order
sector_gva_delta_sorted = sector_gva_delta.sort_values('Output per hour worked', ascending=False)
industry_order = sector_gva_delta_sorted['Industry'].tolist()

# Bar chart with y-axis labels hidden
bar = alt.Chart(sector_gva_delta_sorted).mark_bar().encode(
    y=alt.Y('Industry', sort=industry_order, axis=None),
    x=alt.X('Output per hour worked', axis=alt.Axis(labelFontSize=14, format='%'), scale=alt.Scale(domain=[-0.65, 0.65])),
    tooltip=[alt.Tooltip('Industry', title='Sector'), alt.Tooltip('OPHW_tooltip', title='Output per Hour Worked Change')],
    color=alt.condition(
        alt.datum['Output per hour worked'] > 0,
        alt.value('#36b7b4'),
        alt.value('#e6224b')
    )
)

# Overlay custom y-axis labels with color for Whole Economy
y_labels = alt.Chart(sector_gva_delta_sorted).mark_text(align='right', baseline='middle', dx=-10, fontSize=14).encode(
    y=alt.Y('Industry', sort=industry_order),
    x=alt.value(-0.62),
    text=alt.Text('Industry'),
    color=alt.condition(
        alt.datum['Industry'] == 'Whole Economy',
        alt.value('black'),
        alt.value('gray')
    ),
)

# Text labels for positive change sectors
text_pos = alt.Chart(sector_gva_delta_sorted).mark_text(dx=5, align='left', baseline='middle', fontSize=13).encode(
    y=alt.Y('Industry', sort=industry_order),
    x=alt.X('Output per hour worked'),
    text=alt.Text('OPHW_tooltip'),
    color=alt.value('black'),
).transform_filter(alt.datum['Output per hour worked'] > 0)

# Text labels for negative change sectors
text_neg = alt.Chart(sector_gva_delta_sorted).mark_text(dx=-5, align='right', baseline='middle', fontSize=13).encode(
    y=alt.Y('Industry', sort=industry_order),
    x=alt.X('Output per hour worked'),
    text=alt.Text('OPHW_tooltip'),
    color=alt.value('black'),
).transform_filter(alt.datum['Output per hour worked'] <= 0)

# Add a source text box at the bottom
source_box = alt.Chart(pd.DataFrame({'x': [0], 'y': [-1], 'text': ['Source: ONS']})).mark_text(align='left', fontSize=14, fontStyle='italic', color='gray').encode(
    x=alt.value(-10),
    y=alt.value(435),
    text='text'
)

final_chart = (bar + y_labels + text_pos + text_neg + source_box).properties(
    title=alt.TitleParams(text='UK Productivity change by Sector - 2019 to 2025', subtitle='Output per hour worked calculated as GVA per hour worked', fontSize=16),
    width=400,
    height=400
)

final_chart

alt.LayerChart(...)

In [11]:
# Save to png
final_chart.save('prod_change_by_sector.png', scale_factor=2)
# Save to json
final_chart.save('prod_change_by_sector.json', scale_factor=2)